In [57]:
import numpy as np
from scipy.signal import argrelextrema
import ccxt
import pandas as pd
import numba as nb
from sklearn.preprocessing import StandardScaler, RobustScaler
pd.options.plotting.backend = "plotly"
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [31]:
nb.jit
def create_target(df, long, method='polyfit', polyfit_var='close', pct=0.1):
    if method == 'polyfit':

        trend_list = []
        slope_list = []
        start_list = []
        end_list = []

        index_l = np.arange(long)
        rolling_df = df[polyfit_var].rolling(window=long, min_periods=long)
        for roll in rolling_df:
            if len(roll) < long:
                continue
            slope_array = np.round(np.polyfit(index_l, roll.values, deg=1)[-2], decimals=8)
            slope_list.append(slope_array)
            trend_list.append(np.where(slope_array > 0, 1, np.where(slope_array == 0, 0, -1)).tolist())
            start_list.append(roll.index[0])
            end_list.append(roll.index[long-1])

    y = pd.DataFrame({'trend': trend_list, 'slope': slope_list, 'start_windows': start_list, 'end_windows': end_list})
    return y

In [40]:
def adjusted_sigmoid(x, k=0.5, x0=0):
    """
    Adjusted sigmoid function to map values to the range [-1, 1].
    k controls the steepness of the curve.
    x0 is the midpoint of the sigmoid.
    """
    return 2 / (1 + np.exp(-k * (x - x0))) - 1

In [32]:
def download_1000_candles(market='ETH/USDT', tf='1h'):
    ex = ccxt.binance()
    ohlcv = ex.fetch_ohlcv(market, tf, limit=1001)  # Fetch one extra to ensure 1000 closed candles

    # Build DataFrame
    header = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlcv_df = pd.DataFrame(ohlcv, columns=header)
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms', utc=True)
    # ohlcv_df.set_index('timestamp', inplace=True)

    # Remove the last row to ensure all candles are closed
    ohlcv_df = ohlcv_df.iloc[:-1].copy()

    return ohlcv_df

# download_1000_candles()

In [37]:
candles = download_1000_candles(market='BTC/USDT', tf='30m')
candles

,timestamp,open,high,low,close,volume
0,2023-11-16 18:00:00+00:00,36313.28,36398.61,36080.00,36271.15,1891.76021
1,2023-11-16 18:30:00+00:00,36271.14,36304.00,36105.01,36235.59,724.31767
2,2023-11-16 19:00:00+00:00,36235.59,36265.83,35681.12,35764.52,2495.59038
3,2023-11-16 19:30:00+00:00,35764.52,35935.25,35500.00,35921.01,2767.78923
4,2023-11-16 20:00:00+00:00,35921.01,36059.40,35666.00,35879.64,1671.14740
...,...,...,...,...,...,...
994,2023-12-07 11:00:00+00:00,43134.23,43379.99,43111.10,43355.71,992.41731
995,2023-12-07 11:30:00+00:00,43355.70,43355.71,43153.61,43188.03,619.91860
996,2023-12-07 12:00:00+00:00,43188.03,43188.04,42955.00,43174.45,1736.49593
997,2023-12-07 12:30:00+00:00,43174.44,43409.77,43135.37,43408.00,1031.66352


In [34]:
kernel = 24
min_peaks = argrelextrema(
	candles["low"].values, np.less,
	order=kernel
)
max_peaks = argrelextrema(
	candles["high"].values, np.greater,
	order=kernel
)

candles["extrema"] = 0

for mp in min_peaks[0]:
	candles.at[mp, "extrema"] = -1
for mp in max_peaks[0]:
	candles.at[mp, "extrema"] = 1

candles['extrema'] = candles['extrema'].rolling(
	window=5, win_type='gaussian', center=True).mean(std=0.5)

candles['extrema'].fillna(0, inplace=True)

In [35]:
candles['extrema'].value_counts()

extrema
 0.000000    835
 0.000264     34
 0.106451     34
-0.106451     32
-0.000264     31
 0.786571     17
-0.786571     16
Name: count, dtype: int64

In [43]:
candles = candles.set_index(candles['timestamp'])
target = create_target(candles, 12, method='polyfit', polyfit_var='close')
target = target.set_index('start_windows')
scaled_slope = RobustScaler().fit_transform(target['slope'].values.reshape(-1, 1)).reshape(-1)
target['scaled_slope'] = scaled_slope
target = target.reindex(candles.index)
target['scaled_slope'].fillna(0, inplace=True)

In [59]:
up_threshold = 1
down_threshold = -1

candles['trend'] = 0
candles['trend'] = np.where(target['scaled_slope'] > up_threshold, 1,
				np.where(target['scaled_slope'] < down_threshold, -1, 0))

In [60]:
candles['trend']

timestamp
2023-11-16 18:00:00+00:00    0
2023-11-16 18:30:00+00:00    0
2023-11-16 19:00:00+00:00    1
2023-11-16 19:30:00+00:00    1
2023-11-16 20:00:00+00:00    1
                            ..
2023-12-07 11:00:00+00:00    0
2023-12-07 11:30:00+00:00    0
2023-12-07 12:00:00+00:00    0
2023-12-07 12:30:00+00:00    0
2023-12-07 13:00:00+00:00    0
Name: trend, Length: 999, dtype: int64

In [61]:
candles['trend'].plot()

/Users/rnikitin/miniconda/envs/freqtrade/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [62]:
candles['smooth_trend'] = candles['trend'].rolling(
	window=5, win_type='gaussian', center=True).mean(std=0.5)

In [63]:
candles['smooth_trend'].plot()

/Users/rnikitin/miniconda/envs/freqtrade/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

